In [5]:
%pip install pandas
%pip install scikit-learn


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/d5/2c/734fc9269bdb6768905ac41b82d75264b26925b1e462f4ebf45fe4f17646/scikit_learn-1.5.1-cp312-cp312-macosx_12_0_arm64.whl.metadata
  Using cached scikit_learn-1.5.1-cp312-cp312-macosx_12_0_arm64.whl.metadata (12 kB)
  Obtaining dependency information for scipy>=1.6.0 from https://files.pythonhosted.org/packages/09/70/5e756d3f90fe3fff64d1550db0fa6bb9eb76eedd084c568618f93441f08c/scipy-1.14.0-cp312-cp312-macosx_14_0_arm64.whl.metadata
  Using cached scipy-1.14.0-cp312-cp312-macosx_14_0_arm64.whl.metadata (60 kB)
  Obtaining dependency information for joblib>=1.2.0 from https://files.pythonhosted.org/packages/91/29/df4b9b42f2be0b623cbd5e2140cafcaa2bef0759a00b7b70104dcfe2fb51/joblib-1.4.2-py3-none-any.whl.metadata
  Using c

In [1]:
from openai import OpenAI
import json
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
# Load the updated CSV file
df = pd.read_csv('templates.csv')

# Concatenate the columns
df['combined'] = (
    "component: " + df['component'] +
    "; relevant bank roles: " + df['bank_roles'] +
    "; relevant screen sizes: " + df['screen_sizes']
)

# Print the output
for value in df['combined']:
    print(value)


component: AccountsOverview; relevant bank roles: Bank Manager, Relationship Manager, Sales, Customer Support; relevant screen sizes: iPad, Computer, Phone
component: HighValueTransactions; relevant bank roles: Bank Manager, Relationship Manager, Sales; relevant screen sizes: iPad, Computer, Phone
component: PersonalizedOffers; relevant bank roles: Sales, Relationship Manager; relevant screen sizes: Computer, Phone, iPad
component: RelationshipDetails; relevant bank roles: Bank Manager, Relationship Manager, Customer Support; relevant screen sizes: Computer, iPad
component: QuickLinks; relevant bank roles: Bank Manager, Relationship Manager, Sales, Customer Support; relevant screen sizes: iPad, Computer
component: ESGImpactOverview; relevant bank roles: Bank Manager, Relationship Manager; relevant screen sizes: Computer, iPad
component: InvestmentESGAnalysis; relevant bank roles: Bank Manager, Relationship Manager; relevant screen sizes: Computer, iPad
component: RecentTransactions; re

In [4]:
client = OpenAI(api_key="----")

def get_embedding(text, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

df['ada_embedding'] = df.combined.apply(lambda x: get_embedding(x, model='text-embedding-3-small'))
df.to_csv('embedded_components.csv', index=False)

In [5]:
# Load the embedded components CSV
df = pd.read_csv('embedded_components.csv')
df['ada_embedding'] = df.ada_embedding.apply(eval).apply(np.array)

In [19]:
from openai import OpenAI
import numpy as np

def get_prompt_embedding(prompt_text):
    return np.array(client.embeddings.create(input=[prompt_text], model="text-embedding-3-small").data[0].embedding)


prompt_embedding = get_prompt_embedding(prompt_text)

In [36]:
# Use the embedding model to find the best matching components based on the prompt embedding.
from sklearn.metrics.pairwise import cosine_similarity

# Load the embedded components CSV
df = pd.read_csv('embedded_components.csv')
df['ada_embedding'] = df['ada_embedding'].apply(eval).apply(np.array)

def find_best_components(prompt_embedding, top_n=5):
    similarities = cosine_similarity([prompt_embedding], df['ada_embedding'].tolist())[0]
    top_indices = np.argsort(similarities)[-top_n:]
    return df.iloc[top_indices]

top_components = find_best_components(prompt_embedding)

In [41]:
def generate_layout_config(prompt_text):
    prompt_embedding = get_prompt_embedding(prompt_text)
    top_components = find_best_components(prompt_embedding)
    layout_config = {
        "topSection": {
            "customerInfo": {
                "components": ["Name", "CustomerID", "Status", "RelationshipManager"]
            }
        },
        "mainContentArea": {
            "sections": [
                {"name": row['component'], "cols": row['cols']} for index, row in top_components.iterrows()
            ]
        }
    }
    return layout_config

In [65]:
def generate_layout(prompt_text, layout_config):
    system_message = f"""
    You are HOLO, a highly advanced AI designed to dynamically generate UI layouts for a bank.
    Your goal is to create the most efficient and user-friendly Customer 360 views based on these parameters: customer profile, demographics, behavior, bank user role, and screen size.
    You need to reply in JSON format with the following properties:
    {{
      topSection: {{
        customerInfo: an array of components used for customer information
      }},
      mainContentArea: {{
        layout: either “one-column” or “two-column”
        sections: an array of components displayed in the main content area
      }}
    }}

    The possible values for the mainContentArea sections are:
    AccountOverview, HighValueTransactions, PersonalizedOffers, RelationshipDetails, QuickLinks, ESGImpactOverview, InvestmentESGAnalysis, RecentTransactions,
    LoanDetails,CreditCardSummary, SavingsGoals, BudgetOverview, CustomerInsights, SpendingTrends, FinancialGoals, RetirementPlanning, InsurancePolicies,
    TaxSummary, MortgageDetails, InvestmentPortfolio, CharitableDonations, EmergencyFundStatus, IncomeBreakdown, CreditScore, DebtToIncomeRatio, CreditScoreHistory

    The possible values for the topSection are:
    Name, CustomerID, Status, Age, Gender, Birthday, Address, Phone, Email, Occupation, Income, MaritalStatus, Dependents, CustomerSince, RelationshipManager,
    LastInteraction, PersonalESGScore, CreditScore, PreferredLanguage, RiskProfile, LoyaltyTier, TotalAssets, TotalLiabilities, NetWorth, MembershipExpiry,
    RecentActivity, ProfileCompletion, CustomerSegment, PreferredContactMethod, SpecialNotes

    For example, the response should be:
    {{
      "topSection": {{
        "customerInfo": {{
          "components": ["name", "email", "phone"]
        }}
      }},
      "mainContentArea": {{
        "sections": [
          {{
            "name": "AccountOverview",
            "cols": 8
          }},
          {{
            "name": "HighValueTransactions",
            "cols": 8
          }},
          {{
            "name": "CreditCardSummary",
            "cols": 8
          }},
          {{
            "name": "LoanDetails",
            "cols": 8
          }},
          {{
            "name": "CustomerInsights",
            "cols": 8
          }},
          {{
            "name": "SpendingTrends",
            "cols": 8
          }}
        ] 
      }}
    }}
    """

    prompt=f"""
      { prompt_text }.
      The recommended layout configuration is { layout_config }.
    """

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "system", "content": system_message}, {"role": "user", "content": prompt}],
        max_tokens=250,
        n=1,
        stop=None,
        temperature=0.7,
    )

    print("holo response is ", response.choices[0].message.content)
    enhanced_layout = response.choices[0].message.content
    return enhanced_layout



In [66]:
# Example customer info (as provided in the prompt)
customer_info = {
    'customerId': '00002234',
    'name': 'Anastashia Hoshin',
    'age': 70,
    'address': 'Malolos, Bulacan',
    'birthday': 'Dec 14, 1954',
    'income': 750000,
    'occupation': 'Retired',
    'maritalStatus': 'Married',
    'customerStatus': 'VIP',
    'esgScore': 78,
    'relationshipManager': 'Wilhelm van Astrea',
    'interactionHistory': [
        {
            'date': '2024-07-20',
            'type': 'Email',
            'details': 'Discussed retirement plan options'
        },
        {
            'date': '2024-06-15',
            'type': 'Branch Visit',
            'details': 'Opened new savings account'
        }
    ],
    'accounts': [
        {
            'type': 'savings',
            'balance': 1000,
            'currency': 'PHP'
        },
        {
            'type': 'current',
            'balance': 2000,
            'currency': 'PHP'
        },
        {
            'type': 'loan',
            'balance': 50000,
            'currency': 'PHP',
            'delinquent': 'true'
        }
    ],
    'transactions': [
        {
            'period': '1 month',
            'credits': 10000,
            'debits': 5000
        }
    ],
    'highValueTransactions': [
        {
            'date': '2024-07-01',
            'amount': 100000,
            'type': 'debit'
        }
    ]
}

role="bank manager"
customer_segment=customer_info['customerStatus']
screen_size="iPad screen"

prompt = f"""
Create a layout for a Customer 360 view for a { role } viewing the details of a { customer_segment } in an { screen_size } with this customer info: { customer_info }.
"""

layout_config = generate_layout_config(prompt)
generated_layout = generate_layout(prompt, layout_config)

print("The generated ui is: ", json.dumps(generated_layout, indent=2))



holo response is  ```json
{
  "topSection": {
    "customerInfo": {
      "components": ["Name", "CustomerID", "Status", "RelationshipManager"]
    }
  },
  "mainContentArea": {
    "layout": "two-column",
    "sections": [
      {
        "name": "HighValueTransactions",
        "cols": 6
      },
      {
        "name": "BudgetOverview",
        "cols": 12
      },
      {
        "name": "CreditCardSummary",
        "cols": 6
      },
      {
        "name": "RelationshipDetails",
        "cols": 6
      },
      {
        "name": "CustomerInsights",
        "cols": 12
      }
    ]
  }
}
```
The generated ui is:  "```json\n{\n  \"topSection\": {\n    \"customerInfo\": {\n      \"components\": [\"Name\", \"CustomerID\", \"Status\", \"RelationshipManager\"]\n    }\n  },\n  \"mainContentArea\": {\n    \"layout\": \"two-column\",\n    \"sections\": [\n      {\n        \"name\": \"HighValueTransactions\",\n        \"cols\": 6\n      },\n      {\n        \"name\": \"BudgetOverview\",\n  